In [1]:
import matplotlib.pyplot as plt
import os
import numpy as np
import matplotlib.pyplot as plt
import cc3d
import pandas as pd
import math

from utils.append_df_to_excel import append_df_to_excel

In [ ]:
# #This lets you access the data in a specific sheet for when I try to run this all together
file_path= '/dataNAS/people/anoopai/DODGR/results/T2C_metrics_subregions_pos.xlsx'
data = pd.read_excel(file_path, sheet_name='T2C in sub-regions')

In [3]:
#Finding the number of T2 clusters in each subregion

# Get unique values for each category
subjects = data['Subject'].unique()
knee_types = data['Knee'].unique()
visits = data['Visit'].unique()
intensity_thresholds = data['Intensity Threshold (ms)'].unique()
volume_thresholds = data['Volume Threshold (voxels)'].unique()

#Fill all the empty cells in the Knee column for the controls with NA
data['Knee'] = data['Knee'].fillna('NA')

results = []
# Loop subject, knee type, and visit
for intensity_threshold in intensity_thresholds:
    for volume_threshold in volume_thresholds:
        for subject in subjects:
            for visit in visits:
                for knee_type in knee_types:
                    # Look for current subject, knee, visit, subregion
                    subset = data[(data['Subject'] == subject) & 
                                (data['Knee'] == knee_type) & 
                                (data['Visit'] == visit)] 

                    # Check if subset isn't empty (aka no matching category)
                    if not subset.empty:
                        # groupy helps you pick out a specific subregion + count the T2C labels
                        subregion_counts = subset.groupby('Sub-region Label')['T2C Label'].count()
                        
                        # Convert the groupby function object to df and remove/set the index
                        subregion_counts_df = subregion_counts.reset_index()
                        subregion_counts_df.columns = ['Sub-region Label', 'T2C Num']
                        
                        # Add columns for subject, knee type, and visit
                        subregion_counts_df['Intensity Threshold (ms)'] = intensity_threshold
                        subregion_counts_df['Volume Threshold (voxels)'] = volume_threshold
                        subregion_counts_df['Subject'] = subject
                        subregion_counts_df['Knee'] = knee_type
                        subregion_counts_df['Visit'] = visit
                        
                        #Label 11-15 with corresponding words labels
                        subregion_names = {11:'AN', 12:'MC', 13:'LC', 14:'MP', 15:'LP'}
                        subregion_counts_df['Sub-region'] = subregion_counts_df['Sub-region Label'].map(subregion_names)
                        
                        #Reorder columns
                        subregion_counts_df = subregion_counts_df[['Intensity Threshold (ms)', 'Volume Threshold (voxels)', 'Subject', 'Knee', 'Visit', 'Sub-region Label','Sub-region', 'T2C Num']]
                        results.append(subregion_counts_df)

#put the results together in the new df I made above
cluster_per_subregion_results_df = pd.concat(results, ignore_index=True)

# print(cluster_per_subregion_results_df)

In [4]:
#Finding the % of the subregion covered in T2C.
#Making a copy of the data, repeating many of the steps in the code cell above-- potentially could have simplified this.
# data_copy = data.copy()
# subjects = data_copy['Subject'].unique()
# knee_types = data_copy['Knee'].unique()
# visits = data_copy['Visit'].unique()
# data_copy['Knee'] = data_copy['Knee'].fillna('NA')

results_2 = []
# Loop over each subject, knee type, and visit
for intensity_threshold in intensity_thresholds:
    for volume_threshold in volume_thresholds:
        for subject in subjects:
            for visit in visits:
                for knee_type in knee_types:
                    # Look for current subject, knee, visit, subregion
                    subset = data[(data['Subject'] == subject) & 
                                (data['Knee'] == knee_type) & 
                                (data['Visit'] == visit)] 

                    if not subset.empty:
                        #Sum the # of Voxels for ever cluster in the subregion
                        group_by_subregion = subset.groupby('Sub-region Label').agg({'T2C Voxels': 'sum', 'Sub-region Voxels': 'first'})
                        t2c_percentage_df = group_by_subregion.reset_index()
                        
                        #Find the % covered by cluster
                        t2c_percentage_df['T2C Percent'] = 100 * (t2c_percentage_df['T2C Voxels'] / t2c_percentage_df['Sub-region Voxels'])

                        # Add columns for subject, knee type, and visit
                        t2c_percentage_df['Intensity Threshold (ms)'] = intensity_threshold
                        t2c_percentage_df['Volume Threshold (voxels)'] = volume_threshold
                        t2c_percentage_df['Subject'] = subject
                        t2c_percentage_df['Knee'] = knee_type
                        t2c_percentage_df['Visit'] = visit

                        t2c_percentage_df = t2c_percentage_df[['Intensity Threshold (ms)', 'Volume Threshold (voxels)', 'Subject', 'Knee', 'Visit', 'Sub-region Label', 'T2C Percent']]
                        results_2.append(t2c_percentage_df)
                
# put the results together in the new df I made above
cluster_percentage_results_df = pd.concat(results_2, ignore_index=True)

#Merging the df's from finding T2C count and T2C %
data_all = pd.merge(cluster_per_subregion_results_df, cluster_percentage_results_df, on=['Intensity Threshold (ms)', 'Volume Threshold (voxels)', 'Subject', 'Knee', 'Visit', 'Sub-region Label'], how='left')
# print(data_all)

In [5]:
#Finding the average size (average voxels) of the clusters in that subregion.

results_3 = []
# Loop over each subject, knee type, and visit
for intensity_threshold in intensity_thresholds:
    for volume_threshold in volume_thresholds:
        for subject in subjects:
            for visit in visits:
                for knee_type in knee_types:
                    # Look for current subject, knee, visit, subregion
                    subset = data[(data['Subject'] == subject) & 
                                (data['Knee'] == knee_type) & 
                                (data['Visit'] == visit)] 

                    if not subset.empty:
                        #Making a new df that has the # of voxels of T2 per subregion and number of clusters per subregion
                        group_by_subregion = subset.groupby('Sub-region Label').agg({ 'T2C Voxels': 'sum', 'T2C Label': 'size'})
                        t2c_average_size_df = group_by_subregion.reset_index()

                        #Finding the average voxels size of the clusters in that region
                        t2c_average_size_df['T2C Avg Voxels'] = (t2c_average_size_df['T2C Voxels'] / t2c_average_size_df['T2C Label'])
                        # Calculate the average T2C size in mm^3
                        voxel_dims= (0.3125,  0.3125 , 1.5) # in mm
                        t2c_average_size_df['T2C Avg Size (mm^3)'] = (
                        t2c_average_size_df['T2C Avg Voxels'] *
                        voxel_dims[0] * voxel_dims[1] * voxel_dims[2])
                        
                        # Add columns for subject, knee type, and visit
                        t2c_average_size_df['Intensity Threshold (ms)'] = intensity_threshold
                        t2c_average_size_df['Volume Threshold (voxels)'] = volume_threshold
                        t2c_average_size_df['Subject'] = subject
                        t2c_average_size_df['Knee'] = knee_type
                        t2c_average_size_df['Visit'] = visit

                        t2c_average_size_df = t2c_average_size_df[['Intensity Threshold (ms)', 'Volume Threshold (voxels)','Subject', 'Knee', 'Visit', 'Sub-region Label', 'T2C Avg Voxels', 'T2C Avg Size (mm^3)']]
                        results_3.append(t2c_average_size_df)
                        
# put the results together in the new df I made above
t2c_average_size_results_df = pd.concat(results_3, ignore_index=True)
#print(t2c_average_size_results_df)

#Merge the last data frame with this one
data_all = pd.merge(data_all, t2c_average_size_results_df, on=['Intensity Threshold (ms)', 'Volume Threshold (voxels)','Subject', 'Knee', 'Visit', 'Sub-region Label'], how='left')
# print(data_all)

In [6]:
# # DOES NOT WORK
# 
# This last step is so that every subject, visit, and knee has the subregion labels 11-15

# #Fins the unique combinations of Subject, Knee, Visit
# unique_combinations = data[['Intensity Threshold (ms)', 'Volume Threshold (voxels)', 'Subject', 'Knee', 'Visit']].drop_duplicates()
# subregions = [11, 12, 13, 14, 15]
# all_rows = []

# #Label each unique instance of Subject, Knee, Visit with subregion labels 11-15
# for _, row in unique_combinations.iterrows():
#     for subregion in subregions:
#         all_rows.append([row['Intensity Threshold (ms)'], row['Volume Threshold (voxels)'], row['Subject'], row['Knee'], row['Visit'], subregion])

# # Convert the list to a DataFrame
# all_combinations_df = pd.DataFrame(all_rows, columns=['Intensity Threshold (ms)', 'Volume Threshold (voxels)', 'Subject', 'Knee', 'Visit', 'Sub-region Label'])

# #Mapping all the new 11-15 labels to corresponding subregion name
# subregion_names = {11:'AN', 12:'MC', 13:'LC', 14:'MP', 15:'LP'}
# all_combinations_df['Sub-region'] = all_combinations_df['Sub-region Label'].map(subregion_names)
# #print(all_combinations_df)

# data_all = pd.merge(data_all, all_combinations_df, how='left', on=['Intensity Threshold (ms)', 'Volume Threshold (voxels)', 'Subject', 'Knee', 'Visit', 'Sub-region Label','Sub-region'])
# #Making everything that is a blank cell (because of all the extra labels we added) go to 0.
# data_all.fillna({'Num of T2C': int(0), 'Percent T2C': float(0), 'Avg T2C Voxels': float(0), 'Avg T2C Size (mm^3)': float(0)}, inplace=True)
# print(data_all)

In [ ]:
data_full = pd.DataFrame()

subregions_dict = {'AN': 11, 'MC': 12, 'LC': 13, 'MP': 14, 'LP': 15}
subregions = list(subregions_dict.keys())
sub_all = data_all['Subject'].unique().tolist()
visit_all = data_all['Visit'].unique().tolist()

# Loop through each subject and visit
for sub in sub_all:
    for visit in visit_all:
        # Select data for a single subject and visit
        data_single = data_all[(data_all['Subject'] == sub) & (data_all['Visit'] == visit)]
        subregions_sub = data_single['Sub-region'].unique().tolist()

        # Find missing subregions
        missing_subregions = list(set(subregions) - set(subregions_sub))

        if missing_subregions:
            print(sub, visit, missing_subregions)
            for missing_subregion in missing_subregions:
                # Create a new row for the missing subregion
                data_single_row = pd.DataFrame({
                    'Intensity Threshold (ms)': [data_single['Intensity Threshold (ms)'].unique()[0]],
                    'Volume Threshold (voxels)': [data_single['Volume Threshold (voxels)'].unique()[0]],
                    'Subject': [data_single['Subject'].unique()[0]],
                    'Knee': [data_single['Knee'].unique()[0]],
                    'Visit': [data_single['Visit'].unique()[0]],
                    'Sub-region Label': [subregions_dict[missing_subregion]],
                    'Sub-region': [missing_subregion],
                    'T2C Num': [int(0)],
                    'T2C Percent': [float(0)],
                    'T2C Avg Voxels': [float(0)],
                    'T2C Avg Size (mm^3)': [float(0)]
                })

                # Append the new row to the bottom of data_single
                data_single = pd.concat([data_single, data_single_row], ignore_index=True)
            data_full = pd.concat([data_full, data_single], ignore_index=True)
        else:
            data_full = pd.concat([data_full, data_single], ignore_index=True)
data_full

In [8]:
sheet = 'Sub-region Metrics'
append_df_to_excel(data_full, file_path, sheet)